In [ ]:
print("Notebook is working!")

!pip install langchain chromadb openai pypdf transformers sentence-transformers torch

# Import and setup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from transformers import pipeline
import os
import re

print("All imports successful!")

In [ ]:
"""
PDF Ingestion + Chunking Script with Enhanced Cleaning
------------------------------------------------------
- Loads all PDFs from data directory
- Cleans content to remove bibliography/references/author lists
- Splits text into meaningful chunks
"""

def enhanced_clean_content(text):
    """More aggressive cleaning to remove non-research content"""
    # Remove reference sections
    reference_patterns = [
        r'REFERENCES.*',
        r'BIBLIOGRAPHY.*', 
        r'Reference[s]?.*',
        r'Works Cited.*',
        r'Literature Cited.*',
    ]
    
    for pattern in reference_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.DOTALL)
    
    # Remove citation lines and author lists
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        line_clean = line.strip()
        
        # Skip lines that are likely citations/author lists
        if (re.match(r'^\[\w+\d{2,4}\]', line_clean) or
            re.match(r'^\w+ et al\.', line_clean) or
            re.match(r'^\w+, \w+\.', line_clean) or
            len(line_clean.split()) > 8 and not line_clean.endswith('.') and len(line_clean) < 200 or
            len(line_clean) < 40):
            continue
            
        # Keep substantial content
        if len(line_clean) > 50:
            cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

def ingest_and_clean_pdfs(pdf_dir: str):
    print(f"Looking for PDFs in: {pdf_dir}")

    if not os.path.exists(pdf_dir):
        raise FileNotFoundError(f"Directory not found: {pdf_dir}")

    # Get all PDF files
    pdf_files = [
        os.path.join(pdf_dir, f)
        for f in os.listdir(pdf_dir)
        if f.lower().endswith(".pdf")
    ]

    print(f" Found {len(pdf_files)} PDF files.\n")
    if not pdf_files:
        print(" No PDF files found. Please add PDFs to the data directory.")
        return [], []

    all_docs = []
    failed_files = []

    # Load and clean each PDF
    for path in pdf_files:
        file_name = os.path.basename(path)
        try:
            loader = PyPDFLoader(path)
            docs = loader.load()
            
            # Clean each document
            clean_docs = []
            for doc in docs:
                clean_content = enhanced_clean_content(doc.page_content)
                if len(clean_content.strip()) > 200:  # Only keep substantial content
                    doc.page_content = clean_content
                    clean_docs.append(doc)
            
            all_docs.extend(clean_docs)
            print(f" {file_name}: Loaded {len(clean_docs)} clean pages (from {len(docs)} original)")
            
        except Exception as e:
            print(f" {file_name}: Skipped — {type(e).__name__}: {e}")
            failed_files.append(file_name)

    print(f"\n Total CLEAN pages loaded: {len(all_docs)}")
    if failed_files:
        print(f"Skipped {len(failed_files)} corrupted files: {failed_files}")

    # Split documents into chunks
    print("\n Splitting documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
    )

    chunks = splitter.split_documents(all_docs)
    print(f" Total CLEAN chunks created: {len(chunks)}")

    return all_docs, chunks

In [ ]:
# Execute PDF ingestion
BASE_DIR = os.getcwd()
PDF_DIR = os.path.join(BASE_DIR, "data")
PDF_DIR = os.path.normpath(PDF_DIR)

all_docs, chunks = ingest_and_clean_pdfs(PDF_DIR)
print("\n PDF Ingestion completed successfully!")

In [ ]:
# Create Vector Database with Cleaned Content
print("\n Creating vector database with CLEANED content...")

persist_dir = "data/chroma_db"
os.makedirs(persist_dir, exist_ok=True)

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Create vector store
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)

print(f" Clean Vector DB created at: {persist_dir}")
print(f" Number of clean vectors: {vectordb._collection.count()}")

In [ ]:
# Test the Cleaned Content
print("\n🔍 SAMPLE OF CLEANED CONTENT:")
sample_docs = vectordb.similarity_search("machine learning", k=2)

for i, doc in enumerate(sample_docs):
    print(f"\n--- Sample {i+1} ---")
    print(f"Source: {os.path.basename(doc.metadata.get('source', 'Unknown'))}")
    print(f"Content: {doc.page_content[:600]}...")
    print("-" * 60)

In [ ]:
# -----------------------------------------------
#  Enhanced RAG Query System (with deeper retrieval)
# -----------------------------------------------
print("\n🤖 Setting up Enhanced RAG Query System...")

from transformers import pipeline

# Initialize text generator
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=300,
    temperature=0.3,
    do_sample=True,
)

def smart_rag_query(question, num_docs=8):  #  Increased from 4 → 8 (Option 2)
    """
    Enhanced RAG with better context handling and deeper document retrieval.
    """
    # 🔍 Retrieve more relevant documents (Option 2)
    docs = vectordb.similarity_search(question, k=num_docs)
    
    if not docs:
        return {
            "answer": "No relevant documents found.",
            "sources": []
        }

    #  Build clean context from top documents
    context_parts = []
    for i, doc in enumerate(docs):
        content = doc.page_content
        
        # Clean the content for clarity
        lines = content.split('\n')
        clean_lines = [line.strip() for line in lines if len(line.strip()) > 30]
        clean_content = ' '.join(clean_lines[:5])  # Take top 5 meaningful lines
        
        if clean_content:
            context_parts.append(f"Document {i+1}:\n{clean_content}")

    context = "\n\n".join(context_parts)

    # Create an optimized, synthesis-focused prompt
    prompt = f"""You are an expert research assistant.
Using the following context from multiple academic documents, answer the question clearly.

RESEARCH CONTENT:
{context}

QUESTION: {question}

Please provide a clear and concise explanation based only on the context above:"""

    try:
        # Generate a summarized answer from all sources
        response = generator(prompt, max_length=250)[0]['generated_text']
        
        return {
            "answer": response.strip(),
            "sources": docs
        }

    except Exception as e:
        return {
            "answer": f"Error generating response: {str(e)}",
            "sources": docs
        }


In [ ]:
# Test the RAG System with Multiple Queries
print("\n TESTING RAG SYSTEM")

test_queries = [
    "What are the main machine learning techniques discussed?",
    "What problems in artificial intelligence are being addressed?",
    "What are the key findings about neural networks?",
    "What optimization methods are mentioned for deep learning?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}: {query}")
    print(f"{'='*70}")
    
    result = smart_rag_query(query)
    
    print(" ANSWER:")
    print(result["answer"])
    
    if result["sources"]:
        print(f"\n SOURCES ({len(result['sources'])} documents):")
        unique_sources = set()
        for doc in result["sources"]:
            source = doc.metadata.get('source', 'Unknown')
            filename = os.path.basename(source)
            if filename not in unique_sources:
                unique_sources.add(filename)
                print(f"  - {filename}")
    
    print(f"\n Response generated successfully!")

print(f"\n RAG SYSTEM READY! Total documents in database: {vectordb._collection.count()}")

In [ ]:
# Interactive Query System
print("\n INTERACTIVE QUERY SYSTEM")
print("Type 'quit' to exit, 'sources' to see database info\n")

while True:
    user_query = input("\n Your question: ").strip()
    
    if user_query.lower() == 'quit':
        print(" Goodbye!")
        break
    elif user_query.lower() == 'sources':
        print(f" Database Info:")
        print(f"   - Total vectors: {vectordb._collection.count()}")
        print(f"   - Location: {persist_dir}")
        continue
    elif not user_query:
        continue
    
    print(" Processing...")
    result = smart_rag_query(user_query)
    
    print(f"\nANSWER:")
    print(result["answer"])
    
    if result["sources"]:
        print(f"\n Based on {len(result['sources'])} sources")